<a href="https://colab.research.google.com/github/starq69/z_hacks/blob/master/chroma_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
import duckdb
import chromadb

In [5]:
data = {
    'nome': ['Alice', 'Bob', 'Charlie'],
    'età': [25, 30, 35],
    'altezza': [165, 175, 180],
    'peso': [55, 70, 80]
}
df = pd.DataFrame(data)

# Standardize data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[['età', 'altezza', 'peso']])
df_scaled = pd.DataFrame(df_scaled, columns=['età', 'altezza', 'peso'])
df_scaled['nome'] = df['nome']

In [ ]:
client = chromadb.Client()

collection = client.create_collection('persone')
for row in df_scaled.itertuples(index=False):
    collection.add(
        documents=[row.nome], # Pass the name as a document.
        metadatas={ # Pass other data as metadata.
            'età': row.età,
            'altezza': row.altezza,
            'peso': row.peso,
        },
        ids=[str(row.nome)] # Pass the name as the unique ID (converted to string).
    )

def find_closest_person(input_vector):
    query = {
        'età': input_vector[0],
        'altezza': input_vector[1],
        'peso': input_vector[2]
    }
    # Changed 'top_k' to 'n_results' if you have a ChromaDB version < 0.4
    result = collection.query(query_embeddings=query, n_results=1)
    return result[0]['ids'][0] # Return the ID (name) of the closest person.

input_vector = scaler.transform([[28, 170, 65]])[0]
closest_person = find_closest_person(input_vector)
print(f'La persona più vicina ai valori in input è: {closest_person}')